# Segmenting and Clustering Neighborhoods in Toronto

Note for reviewer : please note the this note book includes both part 1 and part 2 as well.

# Part 1

In [9]:
# importing the libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
# Define the URL where the data should be picked
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(page.content, 'html.parser')

In [11]:
# Getting the data in the table in page into rows
table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

In [12]:
# Creating the Data frame using rows fetched
df = pd.DataFrame(row)
df1 = df[0].str.split('\n', expand=True)
df2 = df1.rename(columns=df1.iloc[0])
df3 = df2.drop(df2.index[0])
df3.head()

,,Postal Code,,Borough,,Neighborhood,
1,,M1A,,Not assigned,,,
2,,M2A,,Not assigned,,,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


# DATA CLEANING

Ignore cells with a borough that is Not assigned.
If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma.

In [13]:
#  Ignore cells with a borough that is Not assigned.
df4 = df3[df3.Borough != 'Not assigned']
df4.head()

,,Postal Code,,Borough,,Neighborhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


In [14]:
i = 0
for i in range(0,df4.shape[0]):
    if df4.iloc[i][2] == 'Not assigned':
        df4.iloc[i][2] = df4.iloc[i][1]
        i = i+1
                                 
result = df4.groupby(['Postal Code','Borough'])['Neighborhood'].apply(', '.join).reset_index()
result.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
result.shape

(103, 3)

# Part 2

In [16]:
# Importing the libraries
import numpy as np
import geocoder

In [17]:
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [20]:
postal_codes = result['Postal Code']    
coords = [get_latlng(postal_code) for postal_code in postal_codes.tolist()]

In [21]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
result['Latitude'] = df_coords['Latitude']
result['Longitude'] = df_coords['Longitude']

In [22]:
result.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284
3,M1G,Scarborough,Woburn,43.771545,-79.218135
4,M1H,Scarborough,Cedarbrae,43.768791,-79.238813
5,M1J,Scarborough,Scarborough Village,43.744203,-79.228725
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.726881,-79.265694
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.713340,-79.284942
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.723538,-79.228353
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.696448,-79.265642
